In [1]:
from deepforest import main
from deepforest import get_data
import os
import matplotlib.pyplot as plt 
import cv2
import pandas as pd
import numpy as np

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [4]:
def display(img,cmap=None):
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap=cmap)

In [ ]:
model = main.deepforest()
model.use_release()

In [ ]:
img_df = model.predict_image(path = '/ofo-share/scratch-amritha/saved_cropped_imgs')

Testing on tiled data (.jpg)
----


In [ ]:
path_to_images = '/ofo-share/scratch-amritha/saved_cropped_imgs'
c = 0 
for filename in os.listdir(path_to_images):
    if filename.endswith('.JPG'):
        c += 1
        img_path = os.path.join(path_to_images, filename)
        img_df = model.predict_image(path = img_path)
        img = cv2.imread(img_path)
    
        for index, row in img_df.iterrows():
            xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
    
        display(img[:,:,::-1])
    if c > 10:  # set this to the number of tile images you want predictions for
        break

Using predict_tile on raster data (.tif)
---

In [9]:
path = "/ofo-share/cv-treedetection-eval_data/photogrammetry-outputs/emerald-point_10a-20230103T2008/ortho.tif"

In [11]:
def detect_trees_tile(path, patch_size, patch_overlap):
    predicted_raster_df = model.predict_tile(path, return_plot = False, patch_size = patch_size, patch_overlap = patch_overlap)
    img = cv2.imread(path)

    for index, row in predicted_raster_df.iterrows():
        xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
        
    cv2.imwrite('/ofo-share/repos-amritha/deepforest/saved_emerald_tile_predictions/emerald_'+str(patch_size)+'_'+str(patch_overlap)+'.tif', img)

In [13]:
detect_trees_tile(path, 400, 0.25)  

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/apallavoor/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

34667 predictions in overlapping windows, applying non-max supression
21561 predictions kept after non-max suppression


In [6]:
raster_path = get_data("/ofo-share/cv-treedetection-eval_data/photogrammetry-outputs/emerald-point_10a-20230103T2008/ortho.tif")
predicted_raster = model.predict_tile(raster_path, return_plot = True, patch_size=300,patch_overlap=0.5) #400

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/apallavoor/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

144071 predictions in overlapping windows, applying non-max supression
52106 predictions kept after non-max suppression


/home/apallavoor/anaconda3/lib/python3.11/site-packages/deepforest/visualize.py:126: UserWarning: No color was provided and the label column is not numeric. Using a single default color.
  warnings.warn("No color was provided and the label column is not numeric. "


In [12]:
t = np.transpose(predicted_raster, (1,2,0))
t.shape

(9790, 9236, 4)

In [4]:
raster_path = get_data("/ofo-share/cv-treedetection-eval_data/photogrammetry-outputs/emerald-point_10a-20230103T2008/ortho.tif")
#predicted_raster_df = model.predict_tile(raster_path, return_plot = False, patch_size=300,patch_overlap=0.5) #400

In [6]:
type(raster_path)

str

In [16]:
predicted_raster_df

,xmin,ymin,xmax,ymax,label,score,image_path
0,2628.0,2138.0,2720.0,2230.0,Tree,0.793714,ortho.tif
1,8598.0,1185.0,8682.0,1273.0,Tree,0.779933,ortho.tif
2,1561.0,1354.0,1658.0,1445.0,Tree,0.772636,ortho.tif
3,3442.0,2.0,3535.0,96.0,Tree,0.763883,ortho.tif
4,2155.0,3312.0,2241.0,3398.0,Tree,0.758132,ortho.tif
...,...,...,...,...,...,...,...
52101,1895.0,8432.0,1921.0,8464.0,Tree,0.100034,ortho.tif
52102,8437.0,6632.0,8457.0,6652.0,Tree,0.100021,ortho.tif
52103,8642.0,2055.0,8684.0,2096.0,Tree,0.100014,ortho.tif
52104,2442.0,9229.0,2463.0,9251.0,Tree,0.100014,ortho.tif


In [17]:
img = cv2.imread("/ofo-share/cv-treedetection-eval_data/photogrammetry-outputs/emerald-point_10a-20230103T2008/ortho.tif")

for index, row in predicted_raster_df.iterrows():
    xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

In [ ]:
plt.imshow(img[:,:,::-1])

In [23]:
cv2.imwrite('predicted.tif', img)

True